In [79]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

In [80]:

df = pd.read_csv("D:\\datasets\\raw\\S&P+500+Stock+Prices+2014-2017.csv\\S&P 500 Stock Prices 2014-2017.csv", parse_dates=['date'])
df = df.sort_values(['symbol','date']).reset_index(drop=True)

df.head()


,symbol,date,open,high,low,close,volume
0,A,2014-01-02,57.10,57.100,56.15,56.21,1916160
1,A,2014-01-03,56.39,57.345,56.26,56.92,1866651
2,A,2014-01-06,57.40,57.700,56.56,56.64,1777472
3,A,2014-01-07,56.95,57.630,56.93,57.45,1463208
4,A,2014-01-08,57.33,58.540,57.17,58.39,2659468


In [81]:
# creating previous close and Daily return 
df['PrevClose'] = df.groupby('symbol')['close'].shift(1)
df['DailyReturn'] = (df['close'] - df['PrevClose']) / df['PrevClose']

In [82]:
df['MA7'] = df.groupby('symbol')['close'].transform(lambda x: x.rolling(7, min_periods=1).mean())
df['MA21'] = df.groupby('symbol')['close'].transform(lambda x: x.rolling(21, min_periods=1).mean())

In [83]:
df['MA7'] = df.groupby('symbol')['close'].transform(lambda x: x.rolling(7, min_periods=1).mean())
df['MA21'] = df.groupby('symbol')['close'].transform(lambda x: x.rolling(21, min_periods=1).mean())


In [84]:
df['CumulativeReturn'] = (
    df.groupby('symbol')['DailyReturn']
      .transform(lambda r: (1 + r.fillna(0)).cumprod() - 1)
)


In [88]:
# Fill missing PrevClose by group
df['PrevClose'] = (
    df.groupby('symbol')['PrevClose']
      .transform(lambda x: x.fillna(df.loc[x.index, 'close']))
)

# Fill missing DailyReturn with 0
df['DailyReturn'] = df['DailyReturn'].fillna(0)

# Forward-fill and backward-fill for open, high, low
for col in ['open', 'high', 'low']:
    df[col] = df[col].ffill().bfill()


In [89]:
df.head()

,symbol,date,open,high,low,close,volume,PrevClose,DailyReturn,MA7,MA21,CumulativeReturn
0,A,2014-01-02,57.10,57.100,56.15,56.21,1916160,56.21,0.000000,56.210,56.210,0.000000
1,A,2014-01-03,56.39,57.345,56.26,56.92,1866651,56.21,0.012631,56.565,56.565,0.012631
2,A,2014-01-06,57.40,57.700,56.56,56.64,1777472,56.92,-0.004919,56.590,56.590,0.007650
3,A,2014-01-07,56.95,57.630,56.93,57.45,1463208,56.64,0.014301,56.805,56.805,0.022060
4,A,2014-01-08,57.33,58.540,57.17,58.39,2659468,57.45,0.016362,57.122,57.122,0.038783


In [90]:
df.to_csv("Stock_data_prepared.csv",index=False)